# Biol 359A  | Regularization and Sparse Regression
## Lecture

### Spring 2022, Week 6
<hr>

Objectives:
-  Regularize complex linear models
-  Evaluate performance of regularized models based on Cross-Validation
-  Discuss the benefits and drawbacks of these methods




In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import sklearn as sk
import urllib.request
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets

warnings.filterwarnings("ignore")
%matplotlib inline

sns.set(rc={'figure.figsize':(11.7,8.27)}) #change figure size
from sklearn import linear_model
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

np.random.seed(0)



TITLE_FONT = 20
LABEL_FONT = 16
TICK_FONT = 16
FIG_SIZE = (10,10)
COLORS= ["#008080","#CA562C"]

sns.set(font_scale=1.5, rc={'figure.figsize':FIG_SIZE}) 
sns.set_style("whitegrid",  {'axes.linewidth': 2, 'axes.edgecolor':'black'})
plt.rc("axes.spines", top=False, right=False)

The following function generates random *in silico* data according to a defined response. Since this is an *in silico* exercise, we know the underlying relationship. The data derives from a 2nd order system.

In [ ]:
def quadratic(x):
    # Default - 2x^2 + 2
    return 2*x**2 + 2

In [ ]:
def generate_noisy_data(function, noise_std, n=10, measurement_std=.2, initial_value=0, x_max=3):
    """
    This function generates noisy data with a certain amount of error applied to the function response.
    The error is normally distributed around the noise_std.
    """
    x = np.linspace(0, x_max, n) 
    x_noise = np.random.normal(0, measurement_std, len(x))
    x += x_noise
    y_noise = np.random.normal(0, noise_std, len(x))
    y = function(x) + initial_value
    y += y_noise
    plt.plot(x, y, 'C0.', label='data')
    x_func = np.linspace(0, max(x)+measurement_std)
    y_func = function(x_func) + initial_value
    plt.plot(x_func, y_func, 'C0--', label='function')
    plt.fill_between(x_func, y_func+noise_std, y_func-noise_std,
                     alpha=0.1)          # Transparency of the fill
    plt.title(r'$ y = 2x^2 + 2$ with noise (std of {})'.format(noise_std), fontsize=TITLE_FONT)
    plt.legend(loc='best')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim(0, max(x)+measurement_std)
    plt.show()
    return x, y


def plot_model(x, y, x_model, y_model, title = ''):
    """
    Plotter function.
    """
    plt.plot(x,y, 'o', label='data')
    plt.plot(x_model, y_model, '--', label='model')
    plt.legend(loc='best')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim(0, max(x))
    plt.title(title)
    plt.show()
    return x, y



### 1. First we need to standardize the data
Procedural note: **standardization** of data (taking the z-score by mean-centering and scaling data by its standard deviation) is required before performing regularization. An example of standardization is shown below. This code walks through the mathematics "under the hood" of `StandardScaler`.

In [ ]:
def zscore_plot(x):
    sns.distplot(x, label='Data')
    mean = np.mean(x)
    std = np.std(x)
    x_zscore = (x - mean)/(std)
    zscored_mean = np.mean(x_zscore)
    zscored_std = np.std(x_zscore)
    sns.distplot(x_zscore, label='Standardized')
    plt.xlabel('x values')
    plt.ylabel('frequencies')
    plt.axvline(mean, color='C0', linestyle='-')
    plt.axvline(mean + std, color='C0', linestyle='--')
    plt.axvline(mean - std, color='C0', linestyle='--')
    plt.axvline(zscored_mean, color='C1', linestyle='-')
    plt.axvline(zscored_mean+zscored_std, color='C1', linestyle='--')
    plt.axvline(zscored_mean-zscored_std, color='C1', linestyle='--')
    plt.legend(loc='best')
    plt.show()
    
x = np.random.normal(6, 3, 300)
zscore_plot(x)

Define necessary functions.

In [ ]:
def polynomial_feature_example(x, y, regularization = None, reg_lambda=1, degrees=6):
    """
    Perform regularization on a polynomial feature set. 
    """
    poly_transform = PolynomialFeatures(degree=degrees, include_bias = False)
    x_poly = poly_transform.fit_transform(x.reshape(-1,1))
    
    #Regularization techniques need to be scaled in order to work properly
    x_scaler = StandardScaler().fit(x_poly)
    y_scaler = StandardScaler().fit(y.reshape(-1,1))
    x_poly_z = x_scaler.transform(x_poly)
    y_z = y_scaler.transform(y.reshape(-1,1))
    
    #Code to perform the model fitting and parameter estimation
    if regularization is None:
        #Least Squares problem
        plt.suptitle('Linear Regression', fontsize=20, fontweight='bold')
        lm_poly = linear_model.LinearRegression(fit_intercept=True)
        lm_poly.fit(x_poly_z,y_z)
        
    elif regularization is 'L1':
        #LASSO problem
        plt.suptitle('LASSO', fontsize=20, fontweight='bold')       
        lm_poly = linear_model.Lasso(alpha = reg_lambda, max_iter=1e8, fit_intercept=True)
        lm_poly.fit(x_poly_z,y_z)    
        
    elif regularization is 'L2':
        #ridge problem
        plt.suptitle('Ridge', fontsize=20, fontweight='bold')
        lm_poly = linear_model.Ridge(alpha = reg_lambda, max_iter=1e5, fit_intercept=True)
        lm_poly.fit(x_poly_z,y_z)
        
    x_model = np.linspace(min(x), max(x), 150).reshape(-1,1)
    x_model_transform = poly_transform.fit_transform(x_model)
    x_model_transform_z = x_scaler.transform(x_model_transform)
    
    
    y_model = lm_poly.predict(x_model_transform_z)*y_scaler.scale_ + y_scaler.mean_
    
    #********************************************************************************
    # Coefficients from scaled model can be transformed back into original units
    # This code is outside the scope of this class and can be ignored. 
    
    unscaled_coefficients = (lm_poly.coef_ * y_scaler.scale_ / x_scaler.scale_).flatten()
    
    poly_terms = [r'$({0:.3f})x ^ {{{1}}}$'.format(coef, i+1) for i, coef in enumerate(unscaled_coefficients)
                 if coef != 0]
    
    unscaled_intercept = lm_poly.intercept_*y_scaler.scale_ + y_scaler.mean_ \
                            - sum(unscaled_coefficients*x_scaler.mean_)
        
    intercept_str = r'${0:.1f} + $'.format(unscaled_intercept[0])
    title =  intercept_str + r'$+$'.join(poly_terms)
    #********************************************************************************
    
    plot_model(x_data, y_data, x_model, y_model, title=title)

### 2. Plot the *in silico* data and the true model. 

In [ ]:
x_data, y_data = generate_noisy_data(quadratic, 1, n=5)

### 3. Add polynomial degrees, and run an MLR.. 

In [ ]:
polynomial_feature_example(x_data, y_data, degrees=6)

### 4. Calculate the LASSO regression coefficients for the *in silico* data generated above. 

In [ ]:
polynomial_feature_example(x_data, y_data, regularization='L1', reg_lambda=0.01)

### 5. Calculate the Ridge regression coefficients for the *in silico* data generated above. 

In [ ]:
polynomial_feature_example(x_data, y_data, regularization='L2', reg_lambda = 0.01)

### 6. Significantly increase the number of observations, *n*, and re-calculate the regression coefficients. 
When the number of samples, *n*, is high, we do not impose the risk of overfitting as it is unlikely we would propose a model with *n*-degrees of freedom when *n>>0*. 


In [ ]:
x_data, y_data = generate_noisy_data(quadratic, 1, n=300)
polynomial_feature_example(x_data, y_data)
polynomial_feature_example(x_data, y_data, regularization='L1', reg_lambda = .1)
polynomial_feature_example(x_data, y_data, regularization='L2', reg_lambda = .1)

### How do we determine an appropriate $\lambda$?

Here is where cross validation comes in. Let's do it with elastic net. Remember that we have a different cost function, with two $\lambda$ parameters. $\lambda_R$ describes the regularization penalty on the $L^2$ norm, and $\lambda_L$ describes the penalty on $L^1$ norm.

In order to work with the Elastic Net software in python, we are going to need to define these in terms of two other parameters. 

$$ \lambda_L =  \alpha * (L_1\text{ratio}) $$
$$ \lambda_R =  \alpha * (1 - L_1\text{ratio}) $$

Where $$ L_1\text{ratio} = \frac{\lambda_L}{\lambda_L + \lambda_R}$$ 

We are going to go with a 80%/20% train/test split. 

In [ ]:
def cross_validation(model, X, y, k=5):
    scores = cross_validate(model, X, y, cv=k,
                            scoring=('r2', 'neg_root_mean_squared_error'),
                            return_train_score=True)
    
    temp_model = model.fit(X, y)
    non_zero = np.count_nonzero(temp_model.coef_)
    return np.mean(scores['train_r2']),np.mean(scores['test_r2']), non_zero

def model_selection(X,y,test=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_scaler = StandardScaler().fit(X_train)
    y_scaler = StandardScaler().fit(y_train.reshape(-1,1))
    X_z = X_scaler.transform(X_train)
    y_z = y_scaler.transform(y_train.reshape(-1,1))
    
    
    X_test_scaler = StandardScaler().fit(X_test)
    y_test_scaler = StandardScaler().fit(y_test.reshape(-1,1))
    X_test_z = X_scaler.transform(X_test)
    y_test_z = y_scaler.transform(y_test.reshape(-1,1))
    
    alpha_dict = {}
    p_dict = {}
    i = 0
    lim = 64
    for alpha in [0, 1e-5, 1e-3, 1e-2, 1e-1, 0.5, 1, 5]:
        alpha_dict[str(alpha)] = {}
        p_dict[str(alpha)] = {}
        for ratio in [0, .1, .5, .7, .9, .95, .99, 1]:
            i += 1
            print(f"{i} of {lim}")
            model = linear_model.ElasticNet(alpha=alpha, l1_ratio=ratio) 

            train_r2, test_r2, non_zero = cross_validation(model, X_z, y_z)
            alpha_dict[str(alpha)][str(ratio)] = f"{train_r2:.4f}/{test_r2:.4f}"
            p_dict[str(alpha)][str(ratio)] = non_zero
            
    cv_df = pd.DataFrame(alpha_dict)
    parameter_df = pd.DataFrame(p_dict)
    cv_df.index.name = "L1 Ratio"
    cv_df.style.set_caption("Alpha")
    parameter_df.index.name = "L1 Ratio"
    parameter_df.style.set_caption("Alpha")    
    return cv_df, parameter_df, X_z, X_test_z, y_z, y_test_z


x_reg, y_reg = make_regression(n_samples=100, n_features=100, 
                               n_informative=5, 
                               n_targets=1, bias=0.2,  
                               noise=6, random_state=504)

print("Columns = alpha")
print("Rows = L1_ratio")
cv_df, parameter_df, X_train, X_test, y_train, y_test = model_selection(x_reg, y_reg)
print("R^2 from TRAIN/VALIDATION:")
cv_df

In [ ]:
print("Number of parameters in each model:")
parameter_df

In [ ]:
def parity_plot(true, pred, r_squared=None, title='', alpha=None, color=None, hue=None):
    """
    plot true vs the predicted data
    inputs: 2 list-like (arrays) data structures
    """
    fig, ax = plt.subplots(1,1,figsize=(10, 8))
    if hue is not None:
        sns.scatterplot(x=true, y=pred, hue=hue)
    else: 
        if color is None: sns.scatterplot(x=true, y=pred)
        else: sns.scatterplot(x=true, y=pred, alpha=alpha, color=color)
    min_value = min(min(true), min(pred))
    max_value = max(max(true), max(pred))
    plt.plot([min_value, max_value],[min_value, max_value], '--', label="parity")
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    sns.despine()
    plt.text(1.01, 0.98, r"$R^2 = {0:.2f}$".format(r_squared),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    plt.title('Parity Plot: {}'.format(title), size=TITLE_FONT)
    plt.legend(loc='best')
    plt.show()    

def test_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    parity_plot(y_train.flatten(), y_train_pred.flatten(), r_squared =model.score(X_train, y_train), title="Training Data", color="grey", alpha=0.5)
    parity_plot(y_test.flatten(), y_test_pred.flatten(), r_squared =model.score(X_test, y_test), title="Test Data", color="blue", alpha=1)

@widgets.interact_manual(alpha=[0, 1e-5, 1e-3, 1e-2, 1e-1, 0.5, 1, 5], l1_ratio=[0, .1, .5, .7, .9, .95, .99, 1])
def build_and_test(alpha, l1_ratio):
    model = linear_model.ElasticNet(alpha=alpha, l1_ratio=l1_ratio) 
    test_model(model, X_train, X_test, y_train, y_test)
